In [ ]:
anomaly_data = np.stack((NDVI , NDRE1 , NDRE2 , NDRE3 , MSAVI ,CCCI , NDWI) , axis = 2)

In [ ]:
anomaly_data.shape

In [ ]:
anomaly_data_reshaped = np.reshape(anomaly_data , [anomaly_data.shape[0]*anomaly_data.shape[1], anomaly_data.shape[2]])
anomaly_data_reshaped.shape

In [ ]:
np.isnan(anomaly_data_reshaped)

In [ ]:
####### Anomaly Detection

inputimg_r = anomaly_data_reshaped

id_nans = np.isnan(inputimg_r[:,0])

inputimg_r_nonans = inputimg_r[~id_nans, :]

COVmatrix = np.cov(inputimg_r_nonans.T)
invCOVmatrix = np.linalg.inv(COVmatrix)

Meanvector = np.mean(inputimg_r_nonans, axis=0)

RXdistances = np.zeros((inputimg_r_nonans.shape[0], 1))

for i in range(inputimg_r_nonans.shape[0]):
  # print(i)
  r = inputimg_r_nonans[i,:].T
  computedRX = np.matmul(np.matmul((r-Meanvector).T, invCOVmatrix), (r-Meanvector))
  RXdistances[i,0] = computedRX

In [ ]:
import numpy as np
mean = np.mean(RXdistances)
std = np.std(RXdistances)

In [ ]:
np.max(RXdistances)

In [ ]:
from scipy.stats import chi2

nbands = inputimg_r.shape[-1]
P = chi2.ppf(0.99, nbands)
#P = mean + 2 * std
finalanomalies = np.zeros_like(RXdistances)

finalanomalies[np.where(RXdistances>P)[0], :]=1

initialimage_r = anomaly_data_reshaped[:,0]
initialimage_r[~id_nans] = np.squeeze(finalanomalies, axis=1)

initialimage = np.reshape(initialimage_r, (dataset.RasterYSize, dataset.RasterXSize))

In [ ]:
import pandas as pd
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
from numpy import where
data = inputimg_r_nonans
model = OneClassSVM(kernel = 'rbf', gamma = 0.001, nu = 0.03).fit(data)
y_pred = model.predict(data)

finalanomalies = np.zeros_like(y_pred)

finalanomalies[np.where(y_pred == -1)[0]]=1

initialimage_r = anomaly_data_reshaped[:,0]
initialimage_r[~id_nans] = (finalanomalies)

initialimage = np.reshape(initialimage_r, (dataset.RasterYSize, dataset.RasterXSize))

In [ ]:
y_pred.shape

In [ ]:
image_kmeans = np.reshape(initialimage,(initialimage.shape[0] * initialimage.shape[1] , 1))
id_nans = np.where(image_kmeans == 1)

input_kmeans = anomaly_data_reshaped[id_nans[0], :]

In [ ]:
input_kmeans.shape

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(input_kmeans)

In [ ]:
kmeans.labels_

initialimage_r[id_nans[0]] = kmeans.labels_ + 1
initialimage = np.reshape(initialimage_r, (dataset.RasterYSize, dataset.RasterXSize))

In [ ]:
kmeans.labels_ 

In [ ]:
plt.imshow(initialimage , cmap = 'jet')

In [ ]:
plt.imshow(NDVI_DATA[0], cmap='jet')

In [ ]:
plt.imshow(NDVI_DATA[4], cmap='jet')